In [10]:
#Compiler Design Individual Assignment

In [37]:
!git clone https://github.com/Esma-esmiz/compiler-design.git

Cloning into 'compiler-design'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 4.34 KiB | 4.34 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [38]:
# Install required packages
!apt-get update
!apt-get install -y flex
!apt-get install -y gcc
!apt-get install -y bison

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 10.2 kB in 1s (7,545 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package 

In [16]:
#1. Program to count the number of characters, words, spaces, end of lines in a given input file.
%%shell

# Create the Lex program
cat << 'EOF' > counter.l
%{
#include <stdio.h>
int char_count = 0;
int word_count = 0;
int space_count = 0;
int line_count = 0;
%}

%%
[a-zA-Z]+    { word_count++; char_count += yyleng; }
[ \t]        { space_count++; char_count++; }
\n           { line_count++; char_count++; }
.            { char_count++; }
%%

int main(int argc, char **argv) {
    if (argc > 1) {
        FILE *file = fopen(argv[1], "r");
        if (!file) {
            perror("Error opening file");
            return 1;
        }
        yyin = file;
    } else {
        yyin = stdin;
    }

    yylex();

    printf("Character count: %d\n", char_count);
    printf("Word count: %d\n", word_count);
    printf("Space count: %d\n", space_count);
    printf("Line count: %d\n", line_count);

    if (argc > 1) {
        fclose(yyin);
    }

    return 0;
}
EOF

# Here is a test text file
cat << 'EOF' > test.txt
This is a test file.
It has multiple lines.
  And some extra spaces.
EOF

# Compile and run the program
flex counter.l
gcc lex.yy.c -o counter -ll
./counter test.txt

Character count: 69
Word count: 13
Space count: 12
Line count: 3


In [17]:
#2. Program to validate email address
%%shell

# Create the Lex program
cat << 'EOF' > email_validator.l
%{
#include <stdio.h>
int valid = 0;
%}

%%
^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$ {
    valid = 1;
    printf("VALID: %s\n", yytext);
}

.+ {
    printf("INVALID: %s\n", yytext);
}

%%

int main() {
    // Read from stdin if no arguments
    if (!isatty(0)) {
        yylex();
    } else {
        printf("Enter email addresses (one per line, Ctrl-D to end):\n");
        yylex();
    }

    return 0;
}


EOF

# Here are some test emails to check the above program
cat << 'EOF' > test_emails.txt
valid@example.com
invalid@.com
user@domain.org
no.at.sign
another.valid@sub.domain.com
EOF

# Compile and run
flex email_validator.l
gcc lex.yy.c -o email_validator -ll
./email_validator < test_emails.txt

VALID: valid@example.com

INVALID: invalid@.com

VALID: user@domain.org

INVALID: no.at.sign

VALID: another.valid@sub.domain.com



In [18]:
#3. Program to validate the bellow password rule
%%shell

# Create the Lex program
cat << 'EOF' > password_validator.l
%{
#include <stdio.h>
#include <string.h>
#include <ctype.h>

int has_upper = 0;
int has_lower = 0;
int has_digit = 0;
int has_special = 0;
int length_ok = 0;

void check_password(char *pwd) {
    for (int i = 0; pwd[i] != '\0'; i++) {
        if (isupper(pwd[i])) has_upper = 1;
        else if (islower(pwd[i])) has_lower = 1;
        else if (isdigit(pwd[i])) has_digit = 1;
        else has_special = 1;
    }
    length_ok = (strlen(pwd) > 8);
}

void print_validation() {
    if (length_ok && has_upper && has_lower && has_digit && has_special) {
        printf("VALID PASSWORD\n");
    } else {
        printf("INVALID PASSWORD. Missing requirements:\n");
        if (!length_ok) printf("- Must be longer than 8 characters\n");
        if (!has_upper) printf("- At least one uppercase letter\n");
        if (!has_lower) printf("- At least one lowercase letter\n");
        if (!has_digit) printf("- At least one digit\n");
        if (!has_special) printf("- At least one special character\n");
    }
}
%}

%%
^.+$ {
    check_password(yytext);
    print_validation();

    // Reset for next password
    has_upper = has_lower = has_digit = has_special = length_ok = 0;
}
%%

int main() {
    printf("Enter passwords to validate (one per line, Ctrl-D to end):\n");
    yylex();
    return 0;
}
EOF

# Create test passwords
cat << 'EOF' > test_passwords.txt
Weak1!
StrongPassword123!
missingupper1!
MissingDigit!
TooShort1!
Perfect@123
EOF

# Compile and run
flex password_validator.l
gcc lex.yy.c -o password_validator -ll
./password_validator < test_passwords.txt

Enter passwords to validate (one per line, Ctrl-D to end):
INVALID PASSWORD. Missing requirements:
- Must be longer than 8 characters

VALID PASSWORD

INVALID PASSWORD. Missing requirements:
- At least one uppercase letter

INVALID PASSWORD. Missing requirements:
- At least one digit

VALID PASSWORD

VALID PASSWORD



In [19]:
#4. Lex program to implement a simple Calculator
%%shell

# Create the Lex program
cat << 'EOF' > calculator.l
%{
#include <stdio.h>
#include <stdlib.h>

int yylex(void);
void yyerror(const char *msg);

int eval(int a, int b, char op) {
    switch(op) {
        case '+': return a + b;
        case '-': return a - b;
        case '*': return a * b;
        case '/':
            if (b == 0) {
                yyerror("Division by zero");
                exit(1);
            }
            return a / b;
        default:
            yyerror("Invalid operator");
            return 0;
    }
}
%}

%%
[0-9]+          { return atoi(yytext); }  // Match integers
[ \t\n]         ;          // Ignore whitespace
[+\-*/]         { return yytext[0]; }  // Match operators
.               { yyerror("Invalid character"); }

%%

int main() {
    printf("Enter simple arithmetic expressions (e.g., 3 + 5):\n");
    printf("Press Ctrl-D to exit.\n\n");

    while(1) {
        printf("> ");
        int a = yylex();
        char op = yylex();
        int b = yylex();

        // Clear remaining input
        while(yylex() != 0);

        int result = eval(a, b, op);
        printf("Result: %d\n", result);
    }

    return 0;
}

void yyerror(const char *msg) {
    fprintf(stderr, "Error: %s\n", msg);
    exit(1);
}
EOF

# Compile and run interactively
flex calculator.l
gcc lex.yy.c -o calculator -ll
echo "To test the calculator interactively, run: ./calculator"
echo "Example session:"
echo -e "> 5 + 3\nResult: 8\n> 10 * 2\nResult: 20\n> 8 / 2\nResult: 4\n> 7 - 3\nResult: 4"


To test the calculator interactively, run: ./calculator
Example session:
> 5 + 3
Result: 8
> 10 * 2
Result: 20
> 8 / 2
Result: 4
> 7 - 3
Result: 4


In [20]:
#5. A networking company wants to validate the URL for their clients. Write a LEXprogram to implement the same.

%%shell

# Create the Lex program
cat << 'EOF' > url_validator.l
%{
#include <stdio.h>
#include <string.h>

int isValid = 0;
char url[1024];

void validate_url() {
    // Basic URL pattern checking
    char *ptr = url;
    int has_protocol = 0;
    int has_domain = 0;

    // Check for protocol (http://, https://, ftp://)
    if (strstr(ptr, "http://") == ptr ||
        strstr(ptr, "https://") == ptr ||
        strstr(ptr, "ftp://") == ptr) {
        has_protocol = 1;
    }

    // Find domain part
    ptr = strstr(ptr, "://");
    if (ptr) ptr += 3;
    else ptr = url;

    // Domain must have at least one dot and valid characters
    char *dot = strchr(ptr, '.');
    if (dot) {
        has_domain = 1;
        while (*ptr) {
            if (!isalnum(*ptr) && *ptr != '.' && *ptr != '-' && *ptr != '_') {
                if (*ptr != '/' && *ptr != ':' && *ptr != '?') {
                    has_domain = 0;
                    break;
                }
            }
            ptr++;
        }
    }

    isValid = (has_protocol && has_domain);
}
%}

%%
^[^\n]+ {
    strncpy(url, yytext, sizeof(url)-1);
    url[sizeof(url)-1] = '\0';
    validate_url();

    if (isValid) {
        printf("VALID URL: %s\n", url);
    } else {
        printf("INVALID URL: %s\n", url);
    }
}
%%

int main() {
    printf("Enter URLs to validate (one per line, Ctrl-D to end):\n");
    yylex();
    return 0;
}
EOF

# Create test URLs
cat << 'EOF' > test_urls.txt
http://www.example.com
https://sub.domain.org/path?query=value
ftp://files.example.net
www.missing.protocol.com
invalid-url
http://localhost:8080
EOF

# Compile and run
flex url_validator.l
gcc lex.yy.c -o url_validator -ll
./url_validator < test_urls.txt

url_validator.l: In function ‘validate_url’:
url_validator.l:31:18: warning: implicit declaration of function ‘isalnum’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
   31 |             if (!isalnum(*ptr) && *ptr != '.' && *ptr != '-' && *ptr != '_') {
      |                  ^~~~~~~
url_validator.l:1:1: note: include ‘<ctype.h>’ or provide a declaration of ‘isalnum’
  +++ |+#include <ctype.h>
    1 | %{
Enter URLs to validate (one per line, Ctrl-D to end):
VALID URL: http://www.example.com

INVALID URL: https://sub.domain.org/path?query=value

VALID URL: ftp://files.example.net

INVALID URL: www.missing.protocol.com

INVALID URL: invalid-url

INVALID URL: http://localhost:8080



In [8]:
#6. Program to validate woldia university student ID number.

%%shell

# Create the Lex program
cat << 'EOF' > studentid_validator.l
%{
#include <stdio.h>
#include <string.h>
#include <ctype.h>

void validate_student_id(char *id) {
    // Check length (exactly 9 characters: WDU + 6 digits)
    if (strlen(id) != 9) {
        printf("INVALID: ID must be exactly 9 characters long\n");
        return;
    }

    // Check prefix (must be WDU)
    if (strncmp(id, "WDU", 3) != 0) {
        printf("INVALID: ID must start with 'WDU'\n");
        return;
    }

    // Check remaining 6 characters are digits
    for (int i = 3; i < 9; i++) {
        if (!isdigit(id[i])) {
            printf("INVALID: Last 6 characters must be digits\n");
            return;
        }
    }

    printf("VALID Woldia University Student ID: %s\n", id);
}
%}

%%
^[A-Za-z0-9]+$ {
    validate_student_id(yytext);
}
%%

int main() {
    printf("Enter Woldia University Student IDs to validate (one per line, Ctrl-D to end):\n");
    printf("Format: WDU followed by 6 digits (e.g., WDU147633)\n\n");
    yylex();
    return 0;
}
EOF

# Here are some test IDs to check the code
cat << 'EOF' > test_ids.txt
WDU147633
WDU000000
WDU123456
wdu123456
WDU12A345
WDU123
WDU1234567
EOF

# Compile and run
flex studentid_validator.l
gcc lex.yy.c -o studentid_validator -ll
./studentid_validator < test_ids.txt

Enter Woldia University Student IDs to validate (one per line, Ctrl-D to end):
Format: WDU followed by 6 digits (e.g., WDU147633)

VALID Woldia University Student ID: WDU147633

VALID Woldia University Student ID: WDU000000

VALID Woldia University Student ID: WDU123456

INVALID: ID must start with 'WDU'

INVALID: Last 6 characters must be digits

INVALID: ID must be exactly 9 characters long

INVALID: ID must be exactly 9 characters long



In [33]:
# First, install the regex library
!pip install regex

import regex

def recognize_scripts(text):
    # Define script patterns with Unicode property escapes
    patterns = {
        'Amharic/Geez': r'\p{Ethiopic}+',
        'Latin': r'\p{Latin}+',
        'Arabic': r'\p{Arabic}+',
        'Chinese': r'\p{Han}+',
        'Devanagari': r'\p{Devanagari}+'
    }

    results = []
    for script, pattern in patterns.items():
        matches = regex.findall(pattern, text)
        for match in matches:
            results.append(f"Recognized {script} script: {match}")

    return '\n'.join(results) if results else "No recognized scripts found"

# Test with sample text
sample_text = "ሰላም Hello 你好 नमस्ते السلام"
print(recognize_scripts(sample_text))

Recognized Amharic/Geez script: ሰላም
Recognized Latin script: Hello
Recognized Arabic script: السلام
Recognized Chinese script: 你好
Recognized Devanagari script: नमस्ते


In [19]:
# 8. Program to recognize amharic/geez character/fidel order liek ሳድስ, ራብዕ , and ሳልስ and all seven.
import unicodedata

# Dictionary mapping Unicode ranges to fidel orders
FIDEL_ORDERS = {
    (0x1200, 0x120F): 1,  # First order (ä)
    (0x1210, 0x121F): 2,  # Second order (u)
    (0x1220, 0x122F): 3,  # Third order (i)
    (0x1230, 0x123F): 4,  # Fourth order (a)
    (0x1240, 0x124F): 5,  # Fifth order (e)
    (0x1250, 0x125F): 6,  # Sixth order (ə)
    (0x1260, 0x126F): 7,  # Seventh order (o)
    # Add more ranges for other consonants
    (0x12A0, 0x12AF): 1,  # አ to ኧ (first order)
    (0x12B0, 0x12BF): 2,  # ከ to ኯ (second order)
    # Continue for all consonant series...
}

def get_fidel_order(char):
    """Determine the fidel order of an Amharic character"""
    code_point = ord(char)
    for (start, end), order in FIDEL_ORDERS.items():
        if start <= code_point <= end:
            return order
    return 0

def analyze_fidel(word):
    """Analyze each character in an Amharic word"""
    print(f"Analyzing: {word}")
    for char in word:
        order = get_fidel_order(char)
        if order > 0:
            print(f"  {char} - Order {order} ({get_order_name(order)})")
        else:
            print(f"  {char} - Not a recognized fidel character")

def get_order_name(order):
    """Get the traditional name for each order"""
    names = {
        1: "First (ä)",
        2: "Second (u)",
        3: "Third (i)",
        4: "Fourth (a)",
        5: "Fifth (e)",
        6: "Sixth (ə)",
        7: "Seventh (o)"
    }
    return names.get(order, "Unknown")

# Test with the words you provided
sample_words = ["ሳድስ", "ራብዕ", "ሳልስ", "ግዕዝ", "ትርጉም"]
for word in sample_words:
    analyze_fidel(word)
    print()

Analyzing: ሳድስ
  ሳ - Order 4 (Fourth (a))
  ድ - Not a recognized fidel character
  ስ - Order 4 (Fourth (a))

Analyzing: ራብዕ
  ራ - Order 3 (Third (i))
  ብ - Order 7 (Seventh (o))
  ዕ - Not a recognized fidel character

Analyzing: ሳልስ
  ሳ - Order 4 (Fourth (a))
  ል - Order 1 (First (ä))
  ስ - Order 4 (Fourth (a))

Analyzing: ግዕዝ
  ግ - Not a recognized fidel character
  ዕ - Not a recognized fidel character
  ዝ - Not a recognized fidel character

Analyzing: ትርጉም
  ት - Not a recognized fidel character
  ር - Order 3 (Third (i))
  ጉ - Not a recognized fidel character
  ም - Order 2 (Second (u))



In [35]:
#9. program to find the length of the longest word

%%shell

# Create the Lex program
cat << 'EOF' > longest_word.l
%{
#include <stdio.h>

int max_length = 0;  // Track longest word length
int current_length = 0;

void check_word() {
    if (current_length > max_length) {
        max_length = current_length;
    }
    current_length = 0;
}
%}

%%
[a-zA-Z]+ {
    current_length = yyleng;  // Get length of matched word
    check_word();
}

[^a-zA-Z]+ {  // Non-word characters
    check_word();
}
%%

int main() {
    yylex();
    printf("Length of longest word: %d\n", max_length);
    return 0;
}
EOF

# Compile and run
flex longest_word.l
gcc lex.yy.c -o longest_word -ll

# Test with sample input
echo "The quickest brown fox jumps over the lazy dog" > test_input.txt
./longest_word < test_input.txt

# Interactive test
# echo "Enter your own text (press Ctrl-D when done):"
# ./longest_word

Length of longest word: 8


In [29]:
#10. program to identify and count positive and negative numbers.

%%shell


# Create the Lex program
cat << 'EOF' > number_counter.l
%{
#include <stdio.h>

int positive_count = 0;
int negative_count = 0;

void process_number(char* num_str) {
    if (num_str[0] == '-') {
        negative_count++;
    } else {
        positive_count++;
    }
}
%}

%%
[-+]?[0-9]+          { process_number(yytext); }  /* Integers */
[-+]?[0-9]+"."[0-9]* { process_number(yytext); }  /* Decimals */
[-+]?"."[0-9]+       { process_number(yytext); }  /* Decimals starting with . */

[ \t\n] ;  /* Ignore whitespace */
.        ;  /* Ignore other characters */
%%

int main() {
    printf("Enter numbers to count (positive and negative):\n");
    yylex();

    printf("\nResults:\n");
    printf("Positive numbers: %d\n", positive_count);
    printf("Negative numbers: %d\n", negative_count);

    return 0;
}
EOF

# Compile and run
flex number_counter.l
gcc lex.yy.c -o number_counter -ll

# Test with sample input
echo "10 -5 3.14 -2.718 0 7 .5 -.25 10 -7" > test_numbers.txt
./number_counter < test_numbers.txt

# # Interactive test
# echo "Enter your own numbers (press Ctrl-D when done):"
# ./number_counter

Enter numbers to count (positive and negative):

Results:
Positive numbers: 6
Negative numbers: 4
